In [ ]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [ ]:
def readData(pickleFiles):
    ''' read pickle data'''
    data = None
    labels = None
    for pf in pickleFiles:
        with open(pf, 'rb') as f:
            save = pickle.load(f)
            if data == None:
                data = save['data']
                labels = save['labels']
            else:
                data = np.concatenate((data, save['data']))
                labels = np.concatenate((labels, save['labels']))
        del save
    return (data, labels)

In [ ]:
train_pickle_files = ['SVHN_train0.pickle', 'SVHN_train1.pickle', 'SVHN_train2.pickle', 'SVHN_train3.pickle', \
                      'SVHN_train4.pickle', 'SVHN_train5.pickle', 'SVHN_train6.pickle']
test_pickle_files = ['SVHN_test0.pickle', 'SVHN_test1.pickle', 'SVHN_test2.pickle']
(train_data, train_labels) = readData(train_pickle_files)
(test_data, test_labels) = readData(test_pickle_files)

In [ ]:
def accuracy(predictions, originals):
    ''' accuracy of our model'''
    out = np.argmax(predictions, 2).T
    ne = 0
    for l in range(out.shape[0]):
    #out = np.argmax(predictions, 2).T
    #print out[l]
    #print batch_labels[l][1:6]
        for i in range(5):
            if (out[l][i] != originals[l][i]):
                #print "%d NOT EQUAL" % (l)
                ne = ne+1
                break
    equ = out.shape[0] - ne
    print "EQUALS %d OUT OF %d" % (equ, out.shape[0])

In [ ]:
# hyper parameters
BATCH_SIZE = 64
IMAGE_SIZE = 32
NUM_CHANNELS = 1
DEPTH1 = 16
DEPTH2 = 32
DEPTH3 = 64
SEED = 66478
OUT_LABELS = 11
ORIG_LABELS = 6
NUM_EPOCHS=10
train_size = train_labels.shape[0]

In [ ]:
graph = tf.Graph()
with graph.as_default():
    train_data_node = tf.placeholder(tf.float32, shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
    train_labels_node = tf.placeholder(tf.int64, shape = (BATCH_SIZE, 5))
    test_data_node = tf.constant(test_data)
    
    conv1_weights = tf.Variable(tf.truncated_normal([5, 5, NUM_CHANNELS, DEPTH1], 
                                stddev = 0.1,
                                seed = SEED,
                                dtype = tf.float32))
    conv1_biases = tf.Variable(tf.constant(0.1, shape=[DEPTH1], dtype=tf.float32))
    conv2_weights = tf.Variable(tf.truncated_normal([5, 5, DEPTH1, DEPTH2],
                                stddev = 0.1,
                                seed = SEED,
                                dtype = tf.float32))
    conv2_biases = tf.Variable(tf.constant(0.1, shape=[DEPTH2], dtype=tf.float32))
    conv3_weights = tf.Variable(tf.truncated_normal([5, 5, DEPTH2, DEPTH3],
                                stddev = 0.1,
                                seed = SEED,
                                dtype = tf.float32))
    conv3_biases = tf.Variable(tf.constant(0.1, shape=[DEPTH3], dtype=tf.float32))
    
    #fully connected layers and biases
    fc1_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc1_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    fc2_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc2_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    fc3_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc3_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    fc4_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc4_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    fc5_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc5_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    def model(data, train=False):
        conv = tf.nn.conv2d(data, conv1_weights, strides=[1,1,1,1], padding='VALID')
        relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
        pool = tf.nn.max_pool(relu, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        conv = tf.nn.conv2d(pool, conv2_weights, strides=[1,1,1,1], padding='VALID')
        relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
        pool = tf.nn.max_pool(relu, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        conv = tf.nn.conv2d(pool, conv3_weights, strides=[1,1,1,1], padding='VALID')
        hidden = tf.nn.relu(tf.nn.bias_add(conv, conv3_biases))
        if train:
            hidden = tf.nn.dropout(hidden, 0.95, seed=SEED)
        shape=hidden.get_shape().as_list()
        reshape=tf.reshape(hidden, [shape[0], shape[1]*shape[2]*shape[3]])
        # fully connected layers
        logits1 = tf.matmul(reshape, fc1_weights) + fc1_biases
        logits2 = tf.matmul(reshape, fc2_weights) + fc2_biases
        logits3 = tf.matmul(reshape, fc3_weights) + fc3_biases
        logits4 = tf.matmul(reshape, fc4_weights) + fc4_biases
        logits5 = tf.matmul(reshape, fc5_weights) + fc5_biases
        return [logits1, logits2, logits3, logits4, logits5]
    train_logits = model(train_data_node, train=True)
    # loss function
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(train_logits[0], train_labels_node[:,0])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(train_logits[1], train_labels_node[:,1])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(train_logits[2], train_labels_node[:,2])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(train_logits[3], train_labels_node[:,3])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(train_logits[4], train_labels_node[:,4]))
            
    batch = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.01, batch*BATCH_SIZE, train_size, 0.95, staircase=True)
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)
    train_prediction = tf.pack([tf.nn.softmax(train_logits[0]),
                                tf.nn.softmax(train_logits[1]),
                                tf.nn.softmax(train_logits[2]),
                                tf.nn.softmax(train_logits[3]),
                                tf.nn.softmax(train_logits[4])])
    test_logits = model(test_data_node, train=False)
    test_prediction = tf.pack([tf.nn.softmax(test_logits[0]),
                                tf.nn.softmax(test_logits[1]),
                                tf.nn.softmax(test_logits[2]),
                                tf.nn.softmax(test_logits[3]),
                                tf.nn.softmax(test_logits[4])])
with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in xrange(int(NUM_EPOCHS*train_size) // BATCH_SIZE):
        offset = (step*BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_data = train_data[offset:(offset + BATCH_SIZE),]
        batch_labels = train_labels[offset:(offset+BATCH_SIZE)]
        feed_dict = {train_data_node:batch_data, train_labels_node:batch_labels}
        _, l, lr,train_out = session.run([optimizer, loss, learning_rate, train_prediction], 
                                               feed_dict=feed_dict)

#        _, l, lr, train_prediction = session.run([optimizer, loss, learning_rate, train_prediction], 
#                                               feed_dict=feed_dict)
        if step%50 == 0:
            accuracy(train_out, batch_labels)
    accuracy(test_prediction.eval(), test_labels)
    test_prediction = test_prediction.eval()
    #print(np.argmax(test_prediction.eval(), 2).T[11])

In [ ]:
with graph.as_default():
    print((test_prediction.shape))
    print(np.argmax(test_prediction, 2).T[11])
    print(np.argmax(test_prediction, 2).T[21])
    

In [ ]:
graph = tf.Graph()

# FIRST ATTEMPT
# Try with fully connected layers
with graph.as_default():
    train_data_node = tf.placeholder(tf.float32, shape = (BATCH_SIZE, IMAGE_SIZE*IMAGE_SIZE))
    train_labels_node = tf.placeholder(tf.int64, shape = (BATCH_SIZE, ORIG_LABELS))
    
    #test_data_node = tf.constant(test_data)
    
    fc1_weights = tf.Variable(tf.truncated_normal([IMAGE_SIZE*IMAGE_SIZE, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc1_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    
    # Training computation.
    logits1 = tf.matmul(train_data_node, fc1_weights) + fc1_biases
    logits2 = tf.matmul(train_data_node, fc1_weights) + fc1_biases
    logits3 = tf.matmul(train_data_node, fc1_weights) + fc1_biases
    logits4 = tf.matmul(train_data_node, fc1_weights) + fc1_biases
    logits5 = tf.matmul(train_data_node, fc1_weights) + fc1_biases
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits1, train_labels_node[:,1])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits2, train_labels_node[:,2])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits3, train_labels_node[:,3])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits4, train_labels_node[:,4])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits5, train_labels_node[:,5]))
    train_prediction = tf.pack([tf.nn.softmax(logits1),
                                tf.nn.softmax(logits2),
                                tf.nn.softmax(logits3),
                                tf.nn.softmax(logits4),
                                tf.nn.softmax(logits5)])     
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.pack([tf.nn.softmax(logits1)])
    #test_prediction = tf.nn.softmax(tf.matmul(test_data_node, fc1_weights) + fc1_biases)

In [ ]:
with tf.Session(graph=graph) as session:
    fc_dataset = train_data.reshape((-1, IMAGE_SIZE * IMAGE_SIZE)).astype(np.float32)
    tf.initialize_all_variables().run()
    print("Initialized")
    #for step in range(num_steps):
    for step in xrange(int(NUM_EPOCHS*train_size) / BATCH_SIZE):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
        # Generate a minibatch.
        batch_data = fc_dataset[offset:(offset + BATCH_SIZE), :]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {train_data_node : batch_data, train_labels_node : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels[:,1:6]))
#print("Validation accuracy: %.1f%%" % accuracy(
#        valid_prediction.eval(), valid_labels))
#  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [ ]:
test_data_node = tf.constant(test_data)

In [ ]:

test_prediction = tf.pack([tf.nn.softmax()])

In [ ]:
graph = tf.Graph()

In [ ]:
test_pred = model(test_data, train=False)

In [ ]:
[test_logits1, test_logits2, test_logits3, test_logits4, test_logits5] = model(test_data_node, train=False)
test_prediction = tf.pack([tf.nn.softmax(test_logits1),
                            tf.nn.softmax(test_logits2),
                            tf.nn.softmax(test_logits3),
                            tf.nn.softmax(test_logits4),
                            tf.nn.softmax(test_logits5)])

In [ ]:
test_prediction = sess.run(test_prediction)

In [ ]:
accuracy(test_prediction, test_labels[:,1:6])

In [ ]:
np.argmax(test_prediction, 2).T[847-1]

In [ ]:
def accuracy(predictions, labels):
    return (np.sum(np.argmax(predictions, 2).T == labels))

In [ ]:
with graph.as_default():
    train_data_node = tf.placeholder(tf.float32, shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
    train_labels_node = tf.placeholder(tf.int64, shape = (BATCH_SIZE, 6))
    
    conv1_weights = tf.Variable(tf.truncated_normal([5, 5, NUM_CHANNELS, DEPTH1], 
                                   stddev = 0.1,
                                   seed = SEED,
                                   dtype = tf.float32))
    conv1_biases = tf.Variable(tf.constant(0.1, shape=[DEPTH1], dtype=tf.float32))
    conv2_weights = tf.Variable(tf.truncated_normal([5, 5, DEPTH1, DEPTH2],
                                   stddev = 0.1,
                                   seed = SEED,
                                   dtype = tf.float32))
    conv2_biases = tf.Variable(tf.constant(0.1, shape=[DEPTH2], dtype=tf.float32))
    conv3_weights = tf.Variable(tf.truncated_normal([5, 5, DEPTH2, DEPTH3],
                                   stddev = 0.1,
                                   seed = SEED,
                                   dtype = tf.float32))
    conv3_biases = tf.Variable(tf.constant(0.1, shape=[DEPTH3], dtype=tf.float32))
    
    #fully connected layers and biases
    fc1_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc1_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    fc2_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc2_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    fc3_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc3_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    fc4_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc4_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    fc5_weights = tf.Variable(tf.truncated_normal([DEPTH3, OUT_LABELS],
                                                 stddev = 0.1,
                                                 seed = SEED, dtype=tf.float32))
    fc5_biases = tf.Variable(tf.constant(0.1, shape=[OUT_LABELS], dtype=tf.float32))
    def model(data, train=False):
        conv = tf.nn.conv2d(data, conv1_weights, strides=[1,1,1,1], padding='VALID')
        relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
        pool = tf.nn.max_pool(relu, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        conv = tf.nn.conv2d(pool, conv2_weights, strides=[1,1,1,1], padding='VALID')
        relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
        pool = tf.nn.max_pool(relu, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        conv = tf.nn.conv2d(pool, conv3_weights, strides=[1,1,1,1], padding='VALID')
        hidden = tf.nn.relu(tf.nn.bias_add(conv, conv3_biases))
        if train:
            hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
        shape=hidden.get_shape().as_list()
        reshape=tf.reshape(hidden, [shape[0], shape[1]*shape[2]*shape[3]])
        # fully connected layers
        logits1 = tf.matmul(reshape, fc1_weights) + fc1_biases
        logits2 = tf.matmul(reshape, fc2_weights) + fc2_biases
        logits3 = tf.matmul(reshape, fc3_weights) + fc3_biases
        logits4 = tf.matmul(reshape, fc4_weights) + fc4_biases
        logits5 = tf.matmul(reshape, fc5_weights) + fc5_biases
        return [logits1, logits2, logits3, logits4, logits5]
    [logits1, logits2, logits3, logits4, logits5] = model(train_data_node, train=True)
    # loss function
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits1, train_labels_node[:,1])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits2, train_labels_node[:,2])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits3, train_labels_node[:,3])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits4, train_labels_node[:,4])) + \
            tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits5, train_labels_node[:,5]))
            
    batch = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.01, batch*BATCH_SIZE, train_size, 0.95, staircase=True)
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)
    train_pred = model(train_data, train=True)
    train_prediction = tf.pack([tf.nn.softmax(train_pred[0]),
                                tf.nn.softmax(train_pred[1]),
                                tf.nn.softmax(train_pred[2]),
                                tf.nn.softmax(train_pred[3]),
                                tf.nn.softmax(train_pred[4])])
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
#        for step in xrange(int(NUM_EPOCHS*train_size) // BATCH_SIZE):
        for step in xrange(1):
            offset = (step*BATCH_SIZE) % (train_size - BATCH_SIZE)
            batch_data = train_data[offset:(offset + BATCH_SIZE),]
            batch_labels = train_labels[offset:(offset+BATCH_SIZE)]
            feed_dict = {train_data_node:batch_data, train_labels_node:batch_labels}
            _, l, lr, predictions = sess.run([optimizer, loss, learning_rate, train_prediction], feed_dict=feed_dict)
            #acc = accuracy(predictions, batch_labels[:,1:6])
            #print('LOSS: %.3f,' % l)
            #print(': %.3f,' % l)

In [ ]:
np.argmax(predictions, 2).T[29-1]

In [ ]:
sess

In [ ]:
sess = tf.Session(graph = graph)

In [ ]:
batch_data = test_data[0:64,]
batch_labels = test_labels[0:64]

In [ ]:
_, l, lr, predictions = sess.run([optimizer, loss, learning_rate, train_prediction],
                                 feed_dict = {train_data_node:batch_data, train_labels_node:batch_labels})

In [ ]:
accuracy(predictions, batch_labels[:,1:6])

In [ ]:
np.sum(np.argmax(predictions, 2).T == batch_labels[:,1:6])

In [ ]:
np.argmax(predictions, 2).T[11]

In [ ]:
np.argmax(predictions, 2).T[0]

In [ ]:
batch_labels[1-1][1:6]

In [ ]:
(np.argmax(predictions, 2).T == train_labels[:,1:6])

In [ ]:
train_labels[26-1][1:6]

In [ ]:
train_labels[offset+1]

In [ ]:
train_labels[:,1].shape

In [ ]:
print train_labels[27]

In [ ]:
print predictions[0][0]

In [ ]:
print predictions[1][10]

In [ ]:
print predictions[0][0]

In [ ]:
print predictions[1][30]

In [ ]:
print predictions[4][0]